# Your data under a different lens: Window functions

On first glance, they look like a watered-down version of the split-applycombine pattern introduced in previous notebook [Pandas UDFs](./9_Pandas_UDF.ipynb). But it contains powerful manipulations in a short and expressive body of code.

Window functions fill a niche between group aggregate (groupBy().agg()) and
group map UDF (groupBy().apply()) transformations, both seen in previous notebook [Pandas UDFs](./9_Pandas_UDF.ipynb). Both rely on partitioning to split the data frame based on a predicate. A group aggregate transformation will yield one record per grouping, while a group map UDF allows for any shape of a resulting data frame; a window function always keeps the dimensions of the data frame intact. Window functions have a secret weapon in the window frame that we define within a partition: it determines which records are included in the application of the function.

Window functions are mostly used for creating new columns, so they leverage
some familiar methods, such as select() and withColumn(). But we will see different approach here.

### Growing and using a simle window function

For this section, we reuse the temperature data set from [RDD & UDF](./8_RDD_n_UDFs.ipynb); the data set contains weather observations for a series of stations, summarized by day. Window functions especially shine when working with time series-like data (e.g., daily observations of temperature) because you can slice the data by day, month, or year and get useful statistics.

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

spark = SparkSession.builder.getOrCreate()

gsod = spark.read.parquet("./data/window/gsod.parquet")

#### Identifying the coldest day of each year, the long way

In this section, we emulate a simple window function through functionality we learned in previous notebooks using the `join()` method.

we start with simple questions to ask our data frame: when and where were the lowest temperature recorded each year? In other words, we want a data frame containing three records, one for each year and showing the station, the date (year, month, day), and the temperature of the coldest day recorded for that year

In [3]:
coldest_temp = gsod.groupby("year").agg(F.min("temp").alias("temp"))
coldest_temp.orderBy("temp").show()

+----+------+
|year|  temp|
+----+------+
|2019|-114.7|
|2017|-114.7|
|2018|-113.5|
+----+------+



This provides the year and the temperature, which are about 40% of the original ask. To get the other three columns (`mo`, `da`, `stn`), we can use a left-semi join on the original table, using the results of `coldest_temp` to resolve the join.

we join `gsod` to `coldest_temp` using a left-semi equi-join on the `year` and `temp` columns. Because `coldest_temp` only contains the coldest temperature for each year, the left semi-join keeps only the records from `gsod` that correspond to that year-temperature pair; this is equivalent to keeping only the records where the temperature was coldest for each year.

In [4]:
coldest_when = gsod.join(
    coldest_temp, how="left_semi", on=["year", "temp"]
).select("stn", "year", "mo", "da", "temp")

coldest_when.orderBy("year", "mo", "da").show()

+------+----+---+---+------+
|   stn|year| mo| da|  temp|
+------+----+---+---+------+
|896250|2017| 06| 20|-114.7|
|896060|2018| 08| 27|-113.5|
|895770|2019| 06| 15|-114.7|
+------+----+---+---+------+



From the above codes we are performing a join between the gsod table and, well, something coming from the gsod table. A self-join, which is when you join a table with itself, is often considered an anti-pattern for data manipulation. While it’s not technically wrong, it can be slow and make the code look more complex than it needs to be.

<img src="images/self_join_table.png">

Fortunately, a window function gives you the same result faster, and with less code
clutter. 

#### Creating and using a simple window function to get the coldest days

We use the `Window` object and parameterize it to split a data frame over column values. We then apply the window over a data frame, using the traditional selector approach.

Similar to split-apply-combine pattern we covered in previous notebooks, we will employ three stages
- Instead of splitting, we’ll partition the data frame.
- Instead of applying, we’ll select values over the window.
- The combine/union operation is implicit (i.e., not explicitly coded) in a window function

Window functions apply over a window of data split according to the values on a column. Each split, called a partition, gets the window function applied to each of its records as if they were independent data frames. The result then gets unioned back into a single data frame.

In [5]:
from pyspark.sql.window import Window

each_year = Window.partitionBy("year")

print(each_year)

A WindowSpec object is nothing more than a blueprint for an eventual window function. We created a window specification called each_year that instructs the window application to split the data frame according to the values in the year column. The real magic happens when you apply the window function to your data
frame.

In [6]:
# Self-Join Approach (repeat)

coldest_when = gsod.join(
    coldest_temp, how="left_semi", on=["year", "temp"]
).select("stn", "year", "mo", "da", "temp")

coldest_when.orderBy("year", "mo", "da").show()

+------+----+---+---+------+
|   stn|year| mo| da|  temp|
+------+----+---+---+------+
|896250|2017| 06| 20|-114.7|
|896060|2018| 08| 27|-113.5|
|895770|2019| 06| 15|-114.7|
+------+----+---+---+------+



Through the `withColumn()` method we define a column, `min_temp`, that collects the minimum of the `temp` column. Now, rather than picking the minimum temperature of the whole data frame, the `min()` is applied over the window specification we defined, using the `over()` method. For each window partition, Spark computes the minimum and then broadcasts the value over each record.

This is an important distinction compared to aggregating functions or UDF: in the
case of a window function, *the number of records in the data frame does not change*. Although `min()` is an aggregate function, since it’s applied with the `over()` method, every record in the window has the minimum value appended. The same would apply for any other aggregate function from `pyspark.sql.functions`, such as `sum()`, `avg()`, `min()`, `max()`, and `count()`.

In [7]:
# Window function

(gsod
.withColumn("min_temp", F.min("temp").over(each_year))
.where("temp = min_temp")
.select("year", "mo", "da", "stn", "temp")
.orderBy("year", "mo", "da")
.show())

+----+---+---+------+------+
|year| mo| da|   stn|  temp|
+----+---+---+------+------+
|2017| 06| 20|896250|-114.7|
|2018| 08| 27|896060|-113.5|
|2019| 06| 15|895770|-114.7|
+----+---+---+------+------+



> **Window functions are just methods on columns**\
Since a window function is applied though a method on a `Column` object, you can also apply them in a `select()`. You can also apply more than one window (or different ones) within the same `select()`. Spark won’t allow you to use a window directly in a `groupby()` or `where()` method, where it’ll spit an `AnalysisException`. If you want to group by or filter according to the result of a window function, “materialize” the column using `select()` or `withColumn()` before using the desired operation

In [8]:
gsod.select(
    "year",
    "mo",
    "da",
    "stn",
    "temp",
    F.min("temp").over(each_year).alias("min_temp"),
).where(
    "temp = min_temp"
).drop(
    "min_temp"
).orderBy(
    "year", "mo", "da"
).show()

+----+---+---+------+------+
|year| mo| da|   stn|  temp|
+----+---+---+------+------+
|2017| 06| 20|896250|-114.7|
|2018| 08| 27|896060|-113.5|
|2019| 06| 15|895770|-114.7|
+----+---+---+------+------+



> **But data frames already have partitions**
Since the beginning of the notebooks, partition has referred to the physical splits of the data on each executor node. Now we are also using partitions with window functions to mean logical splits of the data, which may or may not be equal to the Spark physical ones.

<img src="images/partitions.png">

### Beyond summarizing: Using ranking and analytical functions

There are two families of functions which allow performance of a wider range of operations versus aggregate functions such as `count()`, `sum()`, or `min()`:

- The `ranking` family, which provides information about rank (first, second, all
the way to last), n-tiles, and the ever so useful row number.
- The `analytical` family, which, despite its namesake, covers a variety of behaviors not related to summary or ranking

In [9]:
gsod_light = spark.read.parquet("./data/window/gsod_light.parquet")
gsod_light.show()

+------+----+---+---+----+----------+
|   stn|year| mo| da|temp|count_temp|
+------+----+---+---+----+----------+
|994979|2017| 12| 11|21.3|        21|
|998012|2017| 03| 02|31.4|        24|
|719200|2017| 10| 09|60.5|        11|
|917350|2018| 04| 21|82.6|         9|
|076470|2018| 06| 07|65.0|        24|
|996470|2018| 03| 12|55.6|        12|
|041680|2019| 02| 19|16.1|        15|
|949110|2019| 11| 23|54.9|        14|
|998252|2019| 04| 18|44.7|        11|
|998166|2019| 03| 20|34.8|        12|
+------+----+---+---+----+----------+



#### Ranking functions: Quick, who’s first?

Ranking functions are used for getting the top (or bottom) record for each window partition, or, more generally, for getting an order according to some column’s value.

In [10]:
temp_per_month_asc = Window.partitionBy("mo").orderBy("count_temp")

#### GOLD, SILVER, BRONZE: SIMPLE RANKING USING RANK()

With rank(), each record gets a position based on the value contained in one (or more) columns. Identical values have identical ranks—just like medalists in the Olympics, where the same score/time yields the same rank.

`rank()` takes no parameters since it ranks according to the `orderBy()` method
from the window spec; it would not make sense to order according to one column but
rank according to another

In [11]:
gsod_light.withColumn(
    "rank_tpm", F.rank().over(temp_per_month_asc)
).show()

+------+----+---+---+----+----------+--------+
|   stn|year| mo| da|temp|count_temp|rank_tpm|
+------+----+---+---+----+----------+--------+
|041680|2019| 02| 19|16.1|        15|       1|
|996470|2018| 03| 12|55.6|        12|       1|
|998166|2019| 03| 20|34.8|        12|       1|
|998012|2017| 03| 02|31.4|        24|       3|
|917350|2018| 04| 21|82.6|         9|       1|
|998252|2019| 04| 18|44.7|        11|       2|
|076470|2018| 06| 07|65.0|        24|       1|
|719200|2017| 10| 09|60.5|        11|       1|
|949110|2019| 11| 23|54.9|        14|       1|
|994979|2017| 12| 11|21.3|        21|       1|
+------+----+---+---+----+----------+--------+



The function `rank()` provides nonconsecutive ranks for each record, based on the
value of the ordered value, or the column(s) provided in the `orderBy()` method of
the window spec we call. 

for each window, the lower the count_temp, the lower the rank. When two records have the same ordered value, their ranks are the same. We say that the rank is nonconsecutive because, when you have multiple records that tie for a rank, the next one will be offset by the number of ties. For instance, for `mo` = 03, we have two records with `count_temp` = 12: both are rank 1. The next record (`count_temp` = 24) has a position of 3 rather than 2, because two records tied for the first position.

#### NO TIES WHEN RANKING: USING DENSE_RANK()

What if we want, say, a denser ranking that would allocate consecutive ranks for records? Enter `dense_rank()`. The same principle as rank() applies, where ties share the same rank, but there won’t be any gap between the ranks: 1, 2, 3, and so on. This is practical when you want the second (or third, or any ordinal position) value over a window, rather than the record.

In [13]:
gsod_light.withColumn(
    "rank_tpm", F.dense_rank().over(temp_per_month_asc)
).show()

+------+----+---+---+----+----------+--------+
|   stn|year| mo| da|temp|count_temp|rank_tpm|
+------+----+---+---+----+----------+--------+
|041680|2019| 02| 19|16.1|        15|       1|
|996470|2018| 03| 12|55.6|        12|       1|
|998166|2019| 03| 20|34.8|        12|       1|
|998012|2017| 03| 02|31.4|        24|       2|
|917350|2018| 04| 21|82.6|         9|       1|
|998252|2019| 04| 18|44.7|        11|       2|
|076470|2018| 06| 07|65.0|        24|       1|
|719200|2017| 10| 09|60.5|        11|       1|
|949110|2019| 11| 23|54.9|        14|       1|
|994979|2017| 12| 11|21.3|        21|       1|
+------+----+---+---+----+----------+--------+

